# Some Integrals from Quantum Mechanics

**Rio Agustian Gilang Fernando**

**Advanced Material Science and Nanotechnology**

In [2]:
import numpy as np
from scipy.integrate import quad, dblquad, tplquad

(a) The one-dimensional harmonics oscillator ground state wavefunction is 

$$
\psi_0(a) = N\exp(-q^2/2)
$$

where the displacement from equilibrium, $x$, has been scaled as the dimensionless quantity, $q = (\mu k / \hbar)^{1/4}x, \mu$ is the oscillator mass and $k$ the force constant. Using this scaling for displacement, the classical oscillator turning points are $q_{\pm}=\pm1$, and so the probability of tunneling (i.e, for the oscillator to be found outside the classical limits) is

$$
P(q>q_+) + P(q<q_-) = 2P(q>q_+) = 2\int_{q_+}^{infty} |\psi_0(q)|^2 \mathrm{d}q
$$

This integral can only be evaluated numerically.First, determine the normalization constant from the reuirement that

$$
\braket{\psi_0|\psi_0} = 1 \Rightarrow N^2 \int_{-\infty}^{\infty} |\psi_0(q)|^2 \mathrm{d}q = 1
$$

In [9]:
def func(q):
    return np.exp(-q**2)
I, err = quad(func, -np.inf, np.inf)
N = 1 / np.sqrt(I)
N

0.7511255444649425

In fact, this integral can be evaluated analytically and has the value $I = \sqrt(\pi)$, so $N=\pi^{-1/4}$:

In [12]:
np.pi**(-0.25)

0.7511255444649425

The tunneling probability is then:


In [15]:
I, err = quad(func, 1, np.inf)
Ptunneling = 2 * N**2 * I
Ptunneling

0.1572992070502851

or about 16%.

(b) The two-dimensional particle in a rectengular box has wavefunctions given by:

$$
\psi_{n,m} (x, y) = N \sin{\left(\frac{n\pi x}{L_x}\right)} \, \sin{\left(\frac{m\pi y}{L_y} \right)}
$$

where the particle is confined to a region of zero potential for $0 < x< L_x$ and $0<y<Ly$; and $n=1, 2, 3, ...$ and $m = 1, 2, 3, ...$ are quantum numbers. The normalization constant, $N$, ensures that $\braket{\psi{n,m}|\psi{n,m}} = 1$. To find it, evaluate the integral in the following expression and rearrange for $N$:

$$
N^2 \int_0^{L_y} \int_0^{L_x} \sin{\left(\frac{n\pi x}{L_x}\right)} \, \sin{\left(\frac{m\pi y}{L_y} \right)} \mathrm{d}x \, \mathrm{d}y = 1
$$

Although this integral is not hard to evaluate analytically and gives $N = 2/\sqrt{L_x, L_y}$, we can also use ```dblquad```:

In [34]:
def func(x, y, n, m, Lx, Ly):
    """Return the square of the 2D particle-in-a-box wavefunction."""
    return (np.sin(n* np.pi * x/Lx) * np.sin(m * np.pi * y/Ly))**2

# Example parameters: box dimensions (length units) and quantum numbers.
Lx, Ly = 1.5, 2.5
n, m = 2, 1

# Evaluate the integral of the square of the unnormalized wavefunction
I, unc = dblquad(func, 0, Ly, lambda y: 0, lambda y: Lx, args=(n, m, Lx, Ly))

# The normalization constant is the reciprocal of the square root ot this integral
N = 1 / np.sqrt(I)
N

1.0327955589886444

(the units are those in inverse length). This is the same as the analytical resullt:

In [37]:
2 / np.sqrt(Lx*Ly)

1.0327955589886444

(c) The following electron-electron repulsion integral appears in an approximate treatment of the wavefunction of ahelium atom:

$$
I = \braket{11|11} = \braket{\varphi_1(r_1)\varphi_1(r_1)|\frac{1}{r_{12}}|\varphi_1(r_1)\varphi_1(r_1)}
,$$

where

$$
\varphi(r_i) = \sqrt{\frac{Z^3}{\pi}}e^{-Zr_i}
$$

is the hydrogenic 1s orbital occupied by electron $i$ and the electron-electron distance, $r_{12} = \sqrt{r^2_1-2r_1r_2 \cos{\theta} + r^2_2}$ with $\theta$ the angular separation of the position vectors $\boldsymbol{r_1}$ and $\boldsymbol{r_2}$.

This integral can be evaluated analytically and is found to have the values 5Z/8, but here is a numerical approach.

First note that the integral is over six coordinates: in spherical polar coordinates,$r_1, \theta_1, \phi_1, r_2, \theta_2, \phi_2$:

$$
I = \left(\frac{Z^3}{\pi}\right)^2 \int_0^{2\pi} \int_0^{\pi} \int_0^{\infty} \int_0^{2\pi} \int_0^{\pi} \int_0^{\infty} \, \mathrm{d}r_1\, \mathrm{d} \theta_1\, \mathrm{d} \phi_1\, \mathrm{d} r_2\, \mathrm{d} \theta_2 \, \mathrm{d}\phi_2
$$

The symmetry lets us integrate over both $\phi_1$ and $\phi_2$, and also $\theta_2$ (by letting $\theta=\theta_1$: We need the angular separation of the electrons, so we can fix one and vary the other to obtain all the needed values of $\theta$). This intoduces a factor of $2\pi . 2 \pi .2 = 8\pi^2$ and reduces the integral to one over three coordinates:

$$
I = \left(\frac{Z^3}{\pi}\right)^2 8\pi^2 \int_0^{\infty} \int_0^{\infty}  \int_0^{\pi} \frac{\mathrm{e}^{-2Zr_1}\mathrm{e}^{-2Zr_2}}{r_{12}} \mathrm{d} \theta_1\, \mathrm{d}r_1\,  \, \mathrm{d} r_2
$$

In [71]:
def psi1(r, Z=1):
    return np.sqrt(Z**3 / np.pi) * np.exp(-Z * r)
def func(theta1, r2, r1, Z=1):
    return ((psi1(r1, Z) * psi1(r2, Z) * r1 * r2)**2 * np.sin(theta1)
            / np.sqrt(r1**2 + r2**2 - 2*r1*r2*np.cos(theta1))
           )
Z = 1
I, unc = tplquad(func, 0, np.inf,
                lambda r1: 0, lambda r1: np.inf,
                lambda r1, theta1: 0, lambda r1, theta1: np.pi,
                args=(Z,))
I *= 8 * np.pi**2
I, unc

(0.6250000692531371, 1.489297783794502e-08)

This is, within numerical error, equal to $5/8$.